In [7]:
import folium

class Map:
    def __init__(self, center, zoom):
        self.center = center
        self.zoom = zoom
        self.layers = []
    
    def add_layer(self, layer):
        self.layers.append(layer)
    
    def remove_layer(self, layer):
        self.layers.remove(layer)
    
    def set_zoom(self, zoom):
        self.zoom = zoom
    
    def set_center(self, center):
        self.center = center
    
    def create_marcador(self,lat,lon,titulo):
        marker = folium.Marker(location=[lat, lon], popup=titulo)
        return marker
    
    def show(self):
        m = folium.Map(location=[self.center[0], self.center[1]], zoom_start=self.zoom)
        for layer in self.layers:
            m.add_child(layer)
        return m

## VISUALIZAR MAPA

In [8]:
#VISUALIZAR MAPA
lat=-7.0883544 
lon=-41.4509053

m = Map(center=[lat, lon], zoom=20)

lat=-7.0823544 
lon=-41.4685053
popup='Centro de Picos'
marker = m.create_marcador(lat,lon,popup)
m.add_layer(marker)

lat=-7.0814284
lon=-41.43561163598873
popup='UFPI'
marker = m.create_marcador(lat,lon,popup)
m.add_layer(marker)

m.set_zoom(15)
m.show()

In [9]:
import math

class Coordinate:
    def __init__(self, lat, lon, alt=None):
        self.latitude = lat
        self.longitude = lon
        self.altitude = alt
        
    def distance(self, other, algorithm='euclidean'):
        # calcula a distância entre esta coordenada e outra, usando o algoritmo especificado
        if algorithm == 'euclidean':
            # fórmula da distância euclidiana entre dois pontos
            return math.sqrt((self.latitude - other.latitude)**2 + (self.longitude - other.longitude)**2)
        elif algorithm == 'haversine':
            # fórmula de Haversine para distância entre dois pontos em uma esfera (como a Terra)
            R = 6371  # raio médio da Terra em quilômetros
            d_lat = math.radians(other.latitude - self.latitude)
            d_lon = math.radians(other.longitude - self.longitude)
            lat1 = math.radians(self.latitude)
            lat2 = math.radians(other.latitude)
            a = math.sin(d_lat/2)**2 + math.sin(d_lon/2)**2 * math.cos(lat1) * math.cos(lat2)
            c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
            return R * c
        else:
            raise ValueError(f"Unknown distance algorithm: {algorithm}")


## CALCULA DISTANCIA ENTRE DOIS PONTOS

In [10]:
# criando duas coordenadas
p1 = Coordinate(lat=-22.906847, lon=-43.172897)
p2 = Coordinate(lat=-23.5505, lon=-46.6333)

# calculando a distância de Haversine entre as duas coordenadas
dist_haversine = p1.distance(p2, algorithm='haversine')
print(f"Distância de Haversine: {dist_haversine:.2f} km")

Distância de Haversine: 360.75 km


In [11]:
import requests

class Location:
    def __init__(self, name, address, latitude, longitude):
        self.name = name
        self.address = address
        self.latitude = latitude
        self.longitude = longitude
        self.temperature = None
        self.humidity = None
        self.pressure = None

    def set_weather(self, api_key,lat,lon):
        #url = f"https://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={api_key}"
        url = f'https://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&units=metric&appid={api_key}'
        response = requests.get(url)
        data = response.json()
        
        if 'main' in data:
            self.temperature = data["main"]["temp"]
            self.humidity = data["main"]["humidity"]
            self.pressure = data["main"]["pressure"]
        else:
            self.temperature = None
            self.humidity = None
            self.pressure = None

    def get_coordinates(self):
        return self.latitude, self.longitude


## DADOS METEROLOGICOS DA LOCALIZAÇÃO

In [12]:
# cria um objeto Location para a cidade de Nova York
lat =-23.5505 
lon=-46.6333
nyc = Location("São Paulo", "São Paulo, Brasil", lat,lon)

# obtém as informações meteorológicas para a cidade de Nova York
nyc.set_weather('1fad30734db859e67fe4b73213276fbf',lat ,lon)

# exibe as informações meteorológicas
print(f"Temperatura de {nyc.name}: {nyc.temperature}°C")
print(f"Humidade de {nyc.name}: {nyc.humidity}%")
print(f"Pressão de {nyc.name}: {nyc.pressure} hPa")


Temperatura de São Paulo: 21.97°C
Humidade de São Paulo: 85%
Pressão de São Paulo: 991 hPa


## Converte endereços em coordenadas e vice-versa

In [13]:
import requests
import json

class OSMGeocoder:
    def __init__(self):
        self.base_url = "https://nominatim.openstreetmap.org/search"

    def geocode(self, address):
        params = {
            "q": address,
            "format": "json",
            "addressdetails": 1
        }
        response = requests.get(self.base_url, params=params)
        data = json.loads(response.text)
        if data:
            location = data[0]
            return location["lat"], location["lon"]
        else:
            return None
     
    def reverse_geocode(self, latitude, longitude):
        self.url = "https://nominatim.openstreetmap.org/reverse"
        try:
            response = requests.get(
                self.url,
                params={"format": "json", "lat": latitude, "lon": longitude}
            )
            if response.ok:
                data = response.json()
                address = data.get("address")
                if address:
                    state = address.get("state")
                    if not state:
                        state = address.get("state_district")
                    if not state:
                        state = address.get("region")
                    if not state:
                        state = address.get("county")
                    if not state:
                        state = address.get("city")
                    if not state:
                        state = address.get("town")
                    if not state:
                        state = address.get("village")
                    if not state:
                        state = address.get("hamlet")
                    if not state:
                        state = address.get("island")
                    
                    road = address.get("road")
                    if road:
                        return state, road
                    else:
                        return None, None
                else:
                    return None, None
            else:
                response.raise_for_status()
        except requests.exceptions.HTTPError as error:
            print(f"HTTP error: {error}")
        except requests.exceptions.RequestException as error:
            print(f"Request error: {error}")

## Endereço em Coordenadas

In [14]:
geocoder = OSMGeocoder()
address = "Picos, Piaui, Brasil"
coordinates = geocoder.geocode(address)
if coordinates:
    print(f"A coordenada da {address} é: {coordinates}")
else:
    print(f"Não foi possível localizar coordenadas {address}")

A coordenada da Picos, Piaui, Brasil é: ('-7.0823544', '-41.4685053')


## Coordenadas em Endereço

In [15]:
geocoder = OSMGeocoder()
latitude, longitude = -23.55086445, -46.631580600895475
address = geocoder.reverse_geocode(latitude, longitude)
if address:
    print(f"O endereço da coordenadas ({latitude}, {longitude}) é: {address}")
else:
    print(f"Não foi possível localizar coordenadas ({latitude}, {longitude})")

O endereço da coordenadas (-23.55086445, -46.631580600895475) é: ('São Paulo', 'Rua Silveira Martins')
